In [6]:
import os
import random
import itertools
import re

# 基本的なライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# 描画ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn_analyzer import CustomPairPlot
import graphviz
import pydotplus
from IPython.display import Image
from IPython.display import HTML
from six import StringIO
from ipywidgets import interact, FloatSlider

# データセット分割
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    ShuffleSplit,
    StratifiedShuffleSplit,
)

# 補完
from sklearn.experimental import (
    enable_iterative_imputer,
)  # IterativeImputerをimportするために必要
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# エンコード
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

# サンプリング
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 特徴量選択
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import (
    GenericUnivariateSelect,
    f_classif,
    mutual_info_classif,
    chi2,
)
from boruta import BorutaPy
from libraries.mrmr import mrmr

# models
from lightgbm import LGBMClassifier
import xgboost as xgb


# 学習中
import optuna
from tqdm import tqdm
from sklearn.model_selection import learning_curve, cross_validate, cross_val_score

# 評価指標
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import warnings


# config python file
import config

SEED = config.SEED
# INDEX_MICROARRAY = config.INDEX_MICROARRAY
INDEX_MICROARRAY = 2
from functions import *

fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

# 目的
遺伝子学的分類に基づいた、予後の2値分類を実施する。  
分類はCLAUDIN_SUBTYPEに基づいて実施。  
予後は5年、10年、15年の3つの年次に分けている。Trueで死亡であることに注意すること。

# データ読み込み
読み込み元：
    config.INTERIM_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/claudin_subtype_chi2"

サブタイプ毎のデータを使用 

In [2]:
X_dict = pickle_load(config.INTERIM_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/X_dict.pickle")
y_dict = pickle_load(config.INTERIM_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y_dict.pickle")

# 単一グル―プでのモデルのトレーニング

## ベースモデルの学習結果

各特徴量選択されたdfを用いて基本的な2値分類モデルを学習する

In [3]:
def validate_models(mircroarray_type: str, method: str, sampling_type: str = None):
    for year in range(15, 16, 5):  # 予後年数毎のループ
        prognosis_Xlabel = "X{0:0=2}".format(year)
        prognosis_ylabel = "y{0:0=2}".format(year)

        # compare_bcmsは内部でk分割交差検証を行うので、train_test_splitをしなくて良い
        X_train = X_dict[mircroarray_type][method]["train"][prognosis_Xlabel]
        y_train = y_dict[mircroarray_type][method]["train"][prognosis_ylabel]
        X_val = X_dict[mircroarray_type][method]["val"][prognosis_Xlabel]
        y_val = y_dict[mircroarray_type][method]["val"][prognosis_ylabel]

        # accuracyの表示
        print("----------" * 10)
        print("予後年数：{0:0=2}年:".format(year))
        if accuracy_score(y_train, np.zeros(len(y_train))) >= 0.5:
            score = (
                "0>1".format(year),
                round(accuracy_score(y_train, np.zeros(len(y_train))), 3),
            )
        else:
            score = (
                "0>1".format(year),
                round(accuracy_score(y_train, np.ones(len(y_train))), 3),
            )
        print("accuracyベースライン：", score)
        print("使用特徴量：", X_train.columns)
        print("学習サンプルサイズ：", X_train.shape)
        display("ラベル比率：", y_train.value_counts())
        save_dir = (
            config.TABLES_MODELS_PROGNOSIS_CROSS_DIR
            + "/scores_base-model/{0}/{1}".format(mircroarray_type, method)
        )
        make_dir(save_dir)
        sampling = None
        if sampling_type == "sm":
            sampling = SMOTE(random_state=SEED)
        display(
            compare_bcms(
                X_train,
                y_train,
                X_val,
                y_val,
                over_sampling_class=sampling,
                save_path=save_dir + "/{0:0=2}.csv".format(year),
            )
        )

In [4]:
for k in X_dict.keys():
    print(k)
    validate_models(mircroarray_type=k, method="mrmr", sampling_type=None)
    validate_models(mircroarray_type=k, method="mrmr", sampling_type="sm")
    # validate_models(mircroarray_type=k, method="boruta")

mrna_agilent_microarray_zscores_ref_all_samples
----------------------------------------------------------------------------------------------------
予後年数：15年:
accuracyベースライン： ('0>1', 0.533)
使用特徴量： Index(['RACGAP1', 'PFN4', 'NACAP1', 'RAPGEF5', 'EMR3', 'CXorf57', 'DNAJB7',
       'ENC1', 'PCM1', 'KRTAP12-4', 'STAT5B', 'AI659947', 'TOR1B', 'BP432397',
       'RNU11', 'UHRF1', 'RBBP8', 'USP30', 'AK022229', 'FGF13', 'BE858513',
       'HS3ST1', 'DFNB59', 'GAL3ST4', 'AURKA', 'DB341438', 'C9orf95', 'S100P',
       'ANKAR', 'WDR67', 'GABRB1', 'AW572907', 'HIST1H2BF', 'APLN', 'GTPBP5',
       'SPATA4', 'BG218808', 'AW444974', 'FGD3', 'INTS10', 'CBX7', 'BC033399',
       'TSPYL6', 'TMEM26', 'DA697821', 'MST1', 'IGDCC4', 'UBE2C', 'RAB3B'],
      dtype='object')
学習サンプルサイズ： (865, 49)


'ラベル比率：'

1    461
0    404
Name: OS_15years, dtype: int64

11it [00:02,  5.18it/s]


,acc_train,acc_val,f1_train,f1_val
classifier,,,,
Random Forest,0.870520,0.649485,0.881356,0.679245
Sigmoid SVM,0.757225,0.628866,0.781705,0.678571
AdaBoost,0.854335,0.639175,0.865385,0.672897
RBF SVM,0.915607,0.597938,0.920044,0.648649
Logistic Regression,0.806936,0.608247,0.821772,0.648148
Linear SVM,0.810405,0.587629,0.825532,0.636364
Polynomial SVM,0.966474,0.577320,0.968306,0.630631
Naive Bayes,0.736416,0.567010,0.753247,0.596154
Nearest Neighbors,0.800000,0.567010,0.814973,0.596154


...

----------------------------------------------------------------------------------------------------
予後年数：15年:
accuracyベースライン： ('0>1', 0.533)
使用特徴量： Index(['RACGAP1', 'CYP2D6', 'PTX4', 'BIK', 'ENC1', 'DENND6A', 'STAT5B',
       'RNU11', 'DNAJB7', 'PPIL3', 'UHRF1', 'RBBP8', 'MIR137HG', 'GABRB1',
       'TOR1B', 'MST1', 'NMRK1', 'FGF13', 'HIST1H2BF', 'SPATA4', 'TBC1D31',
       'USP30', 'S100P', 'APLN', 'MTG2', 'GAL3ST4', 'FGD3', 'PJVK', 'INTS10',
       'CBX7', 'FCN2', 'RNASE9', 'SPATA18', 'ANKAR', 'SULT4A1', 'TSPYL6',
       'IGDCC4', 'TMEM26', 'PDK3', 'AURKA', 'PRR21', 'DNAAF1', 'GPR151',
       'GASK1A', 'NLRX1', 'RAB3B', 'HS3ST1', 'PKMYT1', 'CATSPERB'],
      dtype='object')
学習サンプルサイズ： (865, 49)


'ラベル比率：'

1    461
0    404
Name: OS_15years, dtype: int64

11it [00:02,  4.99it/s]


,acc_train,acc_val,f1_train,f1_val
classifier,,,,
Quadratic Discriminant Analysis,0.869848,0.690722,0.864253,0.705882
Logistic Regression,0.780911,0.670103,0.781857,0.686275
Decision Tree,0.815618,0.639175,0.818376,0.672897
RBF SVM,0.903471,0.639175,0.901874,0.666667
Sigmoid SVM,0.732104,0.649485,0.730055,0.666667
Linear SVM,0.799349,0.628866,0.801715,0.660377
Random Forest,0.861171,0.628866,0.862069,0.640000
AdaBoost,0.848156,0.618557,0.845815,0.626263
Naive Bayes,0.749458,0.597938,0.748092,0.613861


## lightGBM

In [11]:
X_val

,RACGAP1,CYP2D6,PTX4,BIK,ENC1,DENND6A,STAT5B,RNU11,DNAJB7,PPIL3,UHRF1,RBBP8,MIR137HG,GABRB1,TOR1B,MST1,NMRK1,FGF13,HIST1H2BF,SPATA4,TBC1D31,USP30,S100P,APLN,MTG2,GAL3ST4,FGD3,PJVK,INTS10,CBX7,FCN2,RNASE9,SPATA18,ANKAR,SULT4A1,TSPYL6,IGDCC4,TMEM26,PDK3,AURKA,PRR21,DNAAF1,GPR151,GASK1A,NLRX1,RAB3B,HS3ST1,PKMYT1,CATSPERB
503,0.2119,1.6529,-1.6855,-0.3577,1.4433,-0.6949,-1.2163,2.0433,0.1040,-0.4316,-0.9287,-0.1893,-0.9244,-0.7795,0.3221,-0.2165,1.2176,0.2797,-0.3739,-0.4375,1.2550,0.0295,0.2200,-0.4103,-0.6604,0.6354,0.4598,0.0183,-1.3312,-1.1823,-0.3141,0.8082,0.5815,-0.4098,0.5083,-1.7614,-0.6782,-0.5668,-0.9551,0.0098,0.8492,0.0100,-0.3438,-0.4686,-1.4428,-0.2313,1.1058,-0.2315,-0.1111
1619,0.1540,1.2383,-1.2778,0.4610,1.1818,-2.2326,-0.9307,-0.6721,-1.3129,0.6635,0.6451,0.3746,1.5337,-0.2018,0.6217,-0.1564,-0.3972,-0.9323,-0.5775,0.6461,0.7516,-0.4811,0.8039,-0.1072,-0.0131,0.5088,1.5258,-0.6589,0.6055,-0.2940,-0.2108,0.6357,0.8730,-0.0041,-0.3072,-0.2047,-0.4924,0.0109,-0.3500,-0.1039,2.0479,0.0358,-0.5422,-1.4356,0.8424,-0.1127,0.8175,-0.0237,0.5653
722,-1.2365,2.4154,0.5176,-2.2447,0.5665,-0.5589,1.6642,-0.6704,0.3184,1.4243,-1.0042,0.4578,-0.3716,0.5093,-0.0479,-0.5220,0.3520,0.2379,-0.6807,-0.5750,-0.4333,-0.3678,-0.9468,0.3422,-2.4815,-0.4230,-0.6113,-0.4163,0.3515,0.5409,-0.8213,-1.4515,0.3249,1.5461,-1.2862,0.3920,3.0093,0.0193,0.2767,-0.9837,0.9380,-0.7992,0.5991,-0.4121,0.3281,-0.6375,1.3134,-1.1243,-0.5217
789,0.6711,1.9597,0.9532,1.8771,0.3071,-0.8478,-1.6705,-0.4652,-0.3583,0.0861,0.0324,-1.6183,1.2612,-0.9678,-0.1074,-0.1820,1.6283,-0.4183,1.2188,-1.0779,4.1171,-0.9964,1.6918,-0.7426,4.0938,-0.0447,-2.1711,-1.6778,-1.8177,-1.7865,0.2300,0.3555,-1.2055,-1.4413,-1.0976,-0.3384,-0.7462,-1.0501,-0.8484,2.4203,-0.9967,-0.7822,-0.1686,-1.2336,0.5812,-0.7414,-1.2036,1.7510,0.6361
497,1.4885,1.5530,0.9744,0.0089,0.1938,1.3090,-1.7878,0.4877,-1.0598,-0.9903,0.6452,-1.5136,-0.4751,-0.8966,0.1184,0.6862,-1.9315,-0.7072,0.3332,-1.1547,1.3148,-0.0365,0.7017,1.0066,-1.0280,0.8721,-1.3742,-0.8728,-2.8240,-0.4807,1.0626,-0.4163,-0.2890,-1.4208,0.4496,1.4724,-0.2414,-0.3148,-0.3273,1.3388,2.2118,-0.2910,0.4340,-0.2061,-1.9057,-0.4447,0.5768,0.6725,-0.7837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,-0.4633,-1.3550,-0.5411,-0.4718,0.9467,-0.4515,0.7681,-0.7827,0.1759,-0.8931,0.2157,0.8962,0.4334,0.6271,0.8521,0.3377,-1.1179,-0.5424,0.0593,-1.0078,-0.4840,0.2536,-0.0460,0.4492,0.1043,0.3842,0.6704,-0.9304,-0.3786,0.7756,0.3579,1.2485,0.5633,-0.5350,-0.9476,0.9232,0.2097,0.2471,0.7001,-0.5191,-0.2644,0.2208,-1.1700,-0.2046,0.3283,0.1034,0.0858,-0.0059,0.1257
808,-0.8221,-0.6471,-1.5889,0.9744,-1.5123,-0.0087,0.6661,-0.4318,-0.1932,-0.1695,-0.8400,-1.4407,-0.8217,0.3155,-0.8692,0.8099,-0.2108,0.3928,-0.6450,-0.7010,1.8789,-0.6556,0.4331,-0.9015,-0.4733,0.2179,0.6511,-0.1976,-1.0250,1.1881,-0.0852,0.7732,-0.7356,0.9526,1.5937,1.5990,-0.5641,0.7393,1.1776,-0.1609,-0.4536,-0.0077,0.6753,0.0342,-0.5892,-0.2515,-0.5434,1.1660,1.2291
10,0.1480,-0.5056,1.0631,1.5842,-0.9057,-1.2109,0.3117,0.8561,0.6465,-0.8311,-0.4831,2.4322,-1.0354,0.9987,0.3688,-0.2068,3.0108,-0.7361,0.2069,1.6876,0.6229,-1.0265,-0.7251,-0.4556,-0.0401,-0.5652,0.5322,-0.1461,0.0848,0.9116,0.8092,-1.2279,2.1052,-0.9137,0.2456,0.6939,-0.5060,0.1345,0.3200,-0.2325,0.0653,1.6664,-0.5129,-0.7258,-0.5341,0.2001,-0.0517,-0.9799,-0.8354
792,-1.1519,1.2391,-0.5695,-0.2012,-0.6040,0.4672,0.9425,-0.2611,0.1630,0.2460,-0.3687,0.0038,-0.1882,2.5137,-0.0245,3.0149,-0.1290,-1.2459,-0.3923,-0.7657,0.7849,-0.1015,1.3743,-0.3543,1.9273,0.9937,0.8515,0.2364,-0.5312,-0.9232,-0.3973,1.1085,-0.2612,1.5100,0.4218,0.5172,-0.2275,1.1722,-0.1942,0.2375,-0.1887,2.8834,-0.6299,0.1466,0.9487,2.3538,-0.7122,0.3911,1.7592


In [12]:
X_train

,RACGAP1,CYP2D6,PTX4,BIK,ENC1,DENND6A,STAT5B,RNU11,DNAJB7,PPIL3,UHRF1,RBBP8,MIR137HG,GABRB1,TOR1B,MST1,NMRK1,FGF13,HIST1H2BF,SPATA4,TBC1D31,USP30,S100P,APLN,MTG2,GAL3ST4,FGD3,PJVK,INTS10,CBX7,FCN2,RNASE9,SPATA18,ANKAR,SULT4A1,TSPYL6,IGDCC4,TMEM26,PDK3,AURKA,PRR21,DNAAF1,GPR151,GASK1A,NLRX1,RAB3B,HS3ST1,PKMYT1,CATSPERB
1179,-1.2052,-0.5416,-0.8235,-1.8149,0.7522,-2.9003,-0.0503,-0.7594,-0.5618,0.4401,-1.0881,-0.6077,0.8412,0.3662,0.3198,2.0915,-2.6340,1.8475,-0.7582,-0.3065,-0.5412,-0.3423,-1.1845,0.4054,1.8695,0.3880,1.7838,1.3993,0.3391,0.3806,1.4520,1.0033,0.2371,0.1927,-0.8906,-0.5021,0.3457,-0.3589,0.3413,-0.4634,-0.5894,0.3656,-0.2322,0.0565,1.1899,-0.7151,-1.0535,-0.8087,0.9016
248,0.4895,-1.9093,0.8939,-0.9732,0.9143,0.8007,-0.9288,0.7774,0.1719,-0.6777,1.3973,-0.2537,2.0394,-0.0619,-0.0492,-1.1150,-1.0804,-0.8170,0.6335,-0.8617,-1.0182,1.3031,1.2350,0.0489,0.1413,2.6842,0.2350,0.0664,-0.1106,-1.5606,1.3172,2.7435,0.9777,-0.1277,-0.1269,1.9955,-0.3947,0.1357,-1.2161,0.1960,-0.4391,-0.5278,-0.3571,0.2335,-0.0419,0.0480,0.8826,0.6808,-0.8725
1010,-0.0800,-2.0482,0.0876,0.7677,0.6644,-0.1323,-0.1755,-0.7657,-0.1460,1.4507,0.1935,1.0147,-0.8546,4.4736,1.0126,-0.5071,-0.0201,-0.7656,0.2892,0.2586,-0.5527,-0.9213,-0.8523,-0.1829,0.7216,-0.8011,0.3776,0.4003,0.5547,0.5586,-0.6819,1.0261,0.1293,1.2210,-1.0506,-0.2384,-0.4237,1.7814,1.3073,0.0931,-1.3498,0.2658,-2.2203,-0.0646,0.2661,-0.5567,2.0942,-0.5139,-0.4338
621,0.9754,-0.7190,0.6587,1.1590,-0.9804,0.9223,-0.7067,0.1708,0.5949,-0.9410,0.3535,0.0891,-1.4304,-0.1579,-0.2842,0.1167,1.0170,-0.3933,0.1584,-1.3519,2.1998,-0.7876,1.7422,0.9884,-2.4693,-0.8476,-0.0737,0.5092,-1.7729,-1.4024,0.6240,-1.4034,-0.6848,-0.3898,-0.2058,1.2852,-0.2067,-0.5263,-0.9119,1.0936,1.4112,0.5773,0.2804,-0.7494,-1.7774,0.7618,1.1356,0.5246,-0.3505
1586,-0.5590,0.9540,-0.2853,0.4371,0.6550,0.3766,-1.1007,-1.1501,-0.9873,0.0706,-0.3272,1.1101,-0.2191,-0.8191,0.4516,1.0980,-0.4956,-0.4220,-0.7367,-0.1836,0.3125,-1.3372,-0.4802,0.0500,0.9575,-1.0240,2.1390,-0.0670,0.5838,-0.6923,-1.4047,1.7806,1.2351,0.7211,0.3853,-0.0810,0.4171,1.0895,-0.3048,-0.6742,0.1007,1.6567,-0.5410,0.9979,0.2817,-0.3792,-0.3749,-0.5507,0.8085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,-1.9819,-0.5004,-0.3520,0.3210,-0.9905,-0.8926,0.5269,0.2579,-0.1180,-0.6313,-1.0042,-0.6637,0.5054,0.9669,-0.4197,1.3788,-0.1660,0.0268,-0.5464,1.6747,-0.6109,0.2531,0.0560,-0.7419,1.8338,-0.4417,0.5887,-0.5377,0.5541,0.0458,0.9950,-0.9646,0.2849,0.3530,1.5453,-0.5614,1.1392,-0.8010,0.6712,-1.1605,0.3506,2.3222,-1.0683,-0.4949,1.9617,-0.3372,-0.7802,-0.6478,-0.7934
846,0.5347,-0.1673,-0.9221,-0.8003,2.0579,-0.1110,-1.3246,-0.4863,-1.8789,-2.6483,-1.0150,-2.3997,0.2032,0.3169,-0.6258,-0.7072,-0.6211,-0.6492,-1.0752,-0.6281,-0.8792,0.6618,1.4519,-0.7643,0.1859,-1.2399,-0.5343,-1.3684,-2.3740,-0.1573,-1.0669,1.2588,-0.6320,-1.1254,-0.4061,-0.9063,0.6077,-1.0246,1.0623,0.5338,-0.7874,-1.0402,0.7516,-0.2885,1.4220,0.2831,0.4587,0.6277,-0.0078
1615,-0.1569,0.3820,-0.8164,-1.4429,1.1329,0.6285,0.4849,1.8331,-1.5506,0.0597,-0.1458,-0.0796,1.2602,-0.2840,-0.4032,-0.2932,0.5094,-0.6146,-0.6994,0.0936,0.3779,-0.3228,-1.4336,-0.2661,-1.7095,-0.5341,0.2066,0.1138,0.5729,1.4639,0.7047,0.3533,-0.3724,0.0305,-0.2723,-0.8876,2.7636,-0.5596,-0.7605,0.5564,-2.5788,-0.9060,-0.8240,-0.7412,1.4015,-0.2924,0.1744,-0.3437,-0.7508
563,1.0143,0.7089,0.1395,2.6576,0.8229,-0.4121,-2.0659,-0.4982,0.3784,0.4169,0.8140,-1.5891,0.3487,-0.4979,-1.9369,-0.7858,-1.2990,-0.9094,1.6310,-1.3859,1.7364,1.0635,1.1233,-0.5893,-0.4916,1.1157,-1.9027,0.5966,-1.4251,-1.1285,0.3671,0.9787,-0.7423,-0.8723,0.1512,0.4548,-0.4727,-0.7810,-0.3583,0.7437,1.7843,-0.7091,0.0713,-0.4332,0.9280,-0.4402,3.6294,0.5938,4.1227


In [13]:
X_train = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["X15"]
y_train = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["y15"]
X_val = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["X15"]
y_val = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["y15"]

params = {
    "max_bin": 100,
    "num_leaves": 30,
    "n_estimators": 800,
    "class_weight": "balanced",
    "random_state": SEED,
}

gbm = LGBMClassifier(**params)
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_val)
show_scores(y_val, y_pred)

accuracy:  0.6391752577319587
precision:  0.6545454545454545
recall:  0.6923076923076923
f1 score:  0.6728971962616823


## xgboost

In [5]:
import xgboost as xgb

X_train = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["X15"]
y_train = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["y15"]
X_val = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["X15"]
y_val = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["y15"]

xgb_train = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns)
xgb_val = xgb.DMatrix(X_val, label=y_val, feature_names=X_val.columns)


param = {
    # 二値分類問題
    "objective": "binary:logistic",
}
clf = xgb.train(param, xgb_train)
y_pred_proba = clf.predict(xgb_val)
y_pred = np.where(y_pred_proba > 0.5, 1, 0)

accuracy_score(y_val, y_pred), f1_score(y_val, y_pred)

[05:23:04] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.6494845360824743, 0.6730769230769231)

### 予測モデル、特徴量数、年数毎の比較

In [5]:
def model2result_list(
    clf: callable,
    X_train: pd.DataFrame(),
    y_train: pd.DataFrame(),
    X_test: pd.DataFrame(),
    y_test: pd.DataFrame(),
    model_name: str,
    year: int,
    k: int,
) -> list():
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    acc = accuracy_score(y_test, pred)
    pre = precision_score(y_test, pred)
    rec = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    return [year, model_name, acc, pre, rec, f1, k]

In [6]:
row = []
fold = 5
for year in tqdm(range(15, 16, 5)):  # 予後年数毎のループ
    prognosis_Xlabel = "X{0:0=2}".format(year)
    prognosis_ylabel = "y{0:0=2}".format(year)
    for k in range(10, 51, 10):
        tmp = None
        X = X_dict["mrna_agilent_microarray_zscores_ref_diploid_samples"]["mrmr"][
            "train"
        ][prognosis_Xlabel]
        y = y_dict["mrna_agilent_microarray_zscores_ref_diploid_samples"]["mrmr"][
            "train"
        ][prognosis_ylabel]

        skf = StratifiedKFold(n_splits=fold, shuffle=True, random_state=SEED)
        for train_index, val_indes in skf.split(X, y):
            X_train, X_val = X.iloc[train_index], X.iloc[val_indes]
            y_train, y_val = y.iloc[train_index], y.iloc[val_indes]

            assert X_train.shape[0] == y_train.shape[0], "train size is incorrect"
            assert X_val.shape[0] == y_val.shape[0], "test size is incorrect"

            # 特徴量選択の実施（mRMR）
            features = mrmr.mrmr_classif(
                X=X_train,
                y=y_train,
                K=k,
                show_progress=False,
            )

            # 特徴量選択で選ばれた特徴量の抽出（訓練、テストデータに適用）
            # list変換するのはfuture warningが出るため
            X_train = X_train[features]
            X_val = X_val[features]

            # スケーリングの実施
            X_train, X_val = transform_std(X_train, X_val)
            # X_train, X_val = transform_norm(X_train, X_val)

            # サンプリングの実装
            # positive_count_train = y_train.value_counts().sort_values()[0]
            rus = RandomUnderSampler(sampling_strategy="all", random_state=SEED)
            # smote = SMOTE(sampling_strategy="all", random_state=SEED)
            X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

            # accuracyの表示
            """
            print("予後年数：{0:0=2}年:".format(year))
            print("学習サンプルサイズ：", X_train.shape)
            display("ラベル比率：", y_train.value_counts())        
            """

            # ここからモデルのtrain、x軸をkとしたacc, f1の推移plot
            # print("Random Forest")
            params = {
                "n_estimators": 500,
                "criterion": "entropy",
                "max_depth": 5,
                "max_leaf_nodes": 20,
                "max_features": "log2",
                "class_weight": "balanced",
                "random_state": SEED,
            }
            rf = RandomForestClassifier(**params)
            tmp = model2result_list(rf, X_train, y_train, X_val, y_val, "rf", year, k)
            row.append(tmp)

            # print("Logistic Regression")
            params = {
                "penalty": "l2",
                "C": 0.5,
                "solver": "saga",
                "class_weight": "balanced",
            }
            lr = LogisticRegression(**params)
            tmp = model2result_list(lr, X_train, y_train, X_val, y_val, "lr", year, k)
            row.append(tmp)

            # print("Support Vector Machine")
            params = {
                "C": 50,
                "kernel": "sigmoid",
                "decision_function_shape": "ovr",
                "class_weight": "balanced",
            }
            svm = SVC(**params)
            tmp = model2result_list(svm, X_train, y_train, X_val, y_val, "svm", year, k)
            row.append(tmp)

    # 結果をdfにまとめている
    df_result = pd.DataFrame(
        row,
        columns=[
            "year",
            "model_name",
            "acc",
            "precision",
            "recall",
            "f1",
            "feature size",
        ],
    )

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:02<00:00, 122.37s/it]


In [7]:
df_result[df_result["model_name"] == "rf"].groupby(
    ["year", "feature size"]
).mean().sort_values("acc", ascending=False)

acc  precision    recall        f1
year feature size                                         
15   40            0.720231   0.734554  0.746166  0.739993
     50            0.720231   0.734554  0.746166  0.739993
     30            0.713295   0.725555  0.743922  0.734229
     20            0.683237   0.694121  0.728822  0.710235
     10            0.677457   0.695627  0.704979  0.699879

In [8]:
df_result[df_result["model_name"] == "lr"].groupby(
    ["year", "feature size"]
).mean().sort_values("acc", ascending=False)

acc  precision    recall        f1
year feature size                                         
15   40            0.732948   0.756673  0.737564  0.746544
     50            0.732948   0.756673  0.737564  0.746544
     30            0.715607   0.739345  0.722394  0.730340
     20            0.695954   0.725919  0.689808  0.707205
     10            0.690173   0.722748  0.681043  0.701143

In [9]:
df_result[df_result["model_name"] == "svm"].groupby(
    ["year", "feature size"]
).mean().sort_values("acc", ascending=False)

acc  precision    recall        f1
year feature size                                         
15   40            0.680925   0.712835  0.674638  0.692703
     50            0.680925   0.712835  0.674638  0.692703
     20            0.669364   0.697698  0.670266  0.683482
     30            0.669364   0.700168  0.670196  0.683615
     10            0.611561   0.653488  0.583567  0.615024

## 最適な特徴量数はいくつなのか実験

特徴量選択数を変化させ、学習曲線を描画する

# subtype毎のベースライン・学習

In [12]:
def validate_models_subtype(
    mircroarray_type: str,
    method: str,
):
    subtypes = [
        "claudin-low",
        "LumA",
        "LumB",
        "Her2",
        "Normal",
        "Basal",
    ]

    for year in range(15, 16, 5):  # 予後年数毎のループ
        print("====={0:0=2}".format(year) * 10)

        for subtype in subtypes:
            prognosis_Xlabel = "X{0:0=2}_{1}".format(year, subtype)
            prognosis_ylabel = "y{0:0=2}_{1}".format(year, subtype)
            X_train = X_dict[mircroarray_type]["claudin_subtype"][method]["train"][
                prognosis_Xlabel
            ]
            y_train = y_dict[mircroarray_type]["claudin_subtype"][method]["train"][
                prognosis_ylabel
            ]
            X_val = X_dict[mircroarray_type]["claudin_subtype"][method]["val"][
                prognosis_Xlabel
            ]
            y_val = y_dict[mircroarray_type]["claudin_subtype"][method]["val"][
                prognosis_ylabel
            ]
            assert X_train.shape[0] == y_train.shape[0], "train size is incorrect"
            assert X_val.shape[0] == y_val.shape[0], "val size is incorrect"
            if X_train.shape[1] == 0:
                print(
                    "**--WARNING: FEATURE NUM is 0! -> subtype {0} train is skipped!--**".format(
                        subtype
                    )
                )
                continue

            # accuracyの表示
            print("----------" * 10)
            print("subtype: ", subtype)
            print("予後年数：{0:0=2}年:".format(year))
            if accuracy_score(y_train, np.zeros(len(y_train))) >= 0.5:
                score = (
                    "0>1".format(year),
                    round(accuracy_score(y_train, np.zeros(len(y_train))), 3),
                )
            else:
                score = (
                    "0>1".format(year),
                    round(accuracy_score(y_train, np.ones(len(y_train))), 3),
                )
            print("accuracyベースライン：", score)
            print("使用特徴量：", X_train.columns)
            print("学習サンプルサイズ：", X_train.shape)
            display("ラベル比率：", y_train.value_counts())

            sm = SMOTE(random_state=SEED)
            display(
                compare_bcms(
                    X_train,
                    y_train,
                    X_val,
                    y_val,
                    over_sampling_class=sm,
                )
            )

In [13]:
for k in X_dict.keys():
    print(k)
    validate_models_subtype(mircroarray_type=k, method="mrmr")
    validate_models_subtype(mircroarray_type=k, method="boruta")

mrna_agilent_microarray_zscores_ref_all_samples
=====15=====15=====15=====15=====15=====15=====15=====15=====15=====15
----------------------------------------------------------------------------------------------------
subtype:  claudin-low
予後年数：15年:
accuracyベースライン： ('0>1', 0.562)
使用特徴量： Index(['BF511322', 'BIRC7', 'ZFP90', 'CHRNB3', 'IL34', 'AA939346', 'AW301012',
       'DB341932', 'MRPL53', 'CHDH', 'DA102929', 'GTPBP5', 'BF447974', 'NAV2',
       'C20orf29', 'NCRNA00160', 'PRPSAP1', 'CD511953', 'STX1B', 'COBL'],
      dtype='object')
学習サンプルサイズ： (89, 20)


'ラベル比率：'

0    50
1    39
Name: OS_15years, dtype: int64

11it [00:00, 41.34it/s]


,acc_train,acc_val,f1_train,f1_val
classifier,,,,
Decision Tree,0.83,0.6,0.852174,0.600000
Quadratic Discriminant Analysis,1.00,0.6,1.000000,0.500000
Linear SVM,1.00,0.5,1.000000,0.285714
Logistic Regression,1.00,0.4,1.000000,0.250000
Polynomial SVM,1.00,0.4,1.000000,0.250000
RBF SVM,1.00,0.4,1.000000,0.250000
Sigmoid SVM,0.95,0.4,0.951456,0.250000
Nearest Neighbors,0.95,0.3,0.952381,0.222222
AdaBoost,1.00,0.2,1.000000,0.000000


...

'ラベル比率：'

1    33
0    25
Name: OS_15years, dtype: int64

11it [00:00, 48.45it/s]


,acc_train,acc_val,f1_train,f1_val
classifier,,,,
Linear SVM,1.000000,0.857143,1.000000,0.888889
Logistic Regression,0.984848,0.857143,0.984615,0.888889
Naive Bayes,0.939394,0.857143,0.937500,0.888889
Nearest Neighbors,0.954545,0.857143,0.952381,0.888889
RBF SVM,0.984848,0.857143,0.984615,0.888889
AdaBoost,1.000000,0.714286,1.000000,0.800000
Random Forest,0.969697,0.714286,0.968750,0.750000
Sigmoid SVM,0.969697,0.714286,0.968750,0.750000
Quadratic Discriminant Analysis,1.000000,0.571429,1.000000,0.727273


**--WARNING: FEATURE NUM is 0! -> this subtype train is skipped!--**
